In [37]:
import requests
import json
import pandas as pd
from datetime import datetime

# Set query (which uses text input to specify ETH wallet addresses)
query = '''query {
  swaps(first:500 orderBy: timestamp, orderDirection: desc){
    id
    timestamp
    pool{
      token0{
        symbol
      }
      token1{
        symbol
      }
    }
    amount0
    amount1
    amountUSD
  }
}
''' 
#% address

# Point to correct subgraph URL
url = 'https://api.thegraph.com/subgraphs/name/benesjan/uniswap-v3-subgraph'
# Make the request
r = requests.post(url, json={'query': query})
#print(r.status_code)
#print(r.text)

# JSON adjustment
json_data = json.loads(r.text)

# extract JSON to convert to a dataframe
df_data = json_data['data']['swaps']
# convert to dataframe
df1 = pd.DataFrame(df_data)  

#print(df1)
#EDA
#df1.to_excel(r'C:\Users\Nehal.Jain\Documents\Nehal_Personal\Project\Blockchain\Dataset.xlsx', sheet_name='Data', index = False)

#Removing NULLS - Data Cleaning
df = df1.dropna()
#print(df)

#print(df.isnull().values.any())
#print(df.isnull().sum().sum())

#Converting from Unix timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')

#Splitting 'pool' column into 'From_Currency' and 'To_Currency'

df[['From_Currency','To_Currency']] = df.pool.apply(
   lambda x: pd.Series(str(x).split(",")))

#Removing extra data from columns From_Currency and To_Currency : Cleaning
df['From_Currency'] = df['From_Currency'].map(lambda x: str(x)[23:-2])
df['To_Currency'] = df['To_Currency'].map(lambda y: str(y)[23:-3])
#print(df)

#Dropping the pool column
df = df.drop(['pool'], axis=1)

#Cleaning and re-ordering of data
df = df.rename(columns={'amount0': 'From_Currency_Amount', 'amount1': 'To_Currency_Amount', 'amountUSD': 'Amount_USD', 'id': 'Wallet_Address','timestamp': 'Date_Time'})
df [['Wallet_Address','From_Currency_Amount','From_Currency','To_Currency_Amount','To_Currency','Amount_USD','Date_Time']]



,Wallet_Address,From_Currency_Amount,From_Currency,To_Currency_Amount,To_Currency,Amount_USD,Date_Time
0,0xe1b3a91f6219f3068f8e86e9c929fa1e99eda6dcb66b...,0.04,WETH,-1115,ATRI,83.50092777951262246989029047240915,2021-07-10 17:53:36
1,0xcf085002b736bd5f351579ff66ae0a0d97d59530ea56...,-8049.686661727311107999,INST,27.139106495285800358,WETH,56653.51428658403838649498441661525,2021-07-10 17:53:36
2,0xcf085002b736bd5f351579ff66ae0a0d97d59530ea56...,56707.278637501302257695,DAI,-27.139106495285800358,WETH,56700.06062681504214515910108930235,2021-07-10 17:53:36
3,0xca865a5bc11cc9842ba18a281bb18a80738187d30946...,-33325.55853,USDC,15.950430092684421362,WETH,33311.22565526758366570863897320963,2021-07-10 17:53:36
4,0xa87c7d8f6570c0e6d87ed5e30f87c65a9a0572489250...,7686.699146309436710399,INST,-25.766168578577639774,WETH,53787.4745408639725572352986637784,2021-07-10 17:53:36
5,0x9b2446aab07e5f2b1eb9689db2e1b6f5db28a0c3f1e8...,-17684.975340377415507612,DAI,8.468597217059165003,WETH,17687.81677871897836093385912780833,2021-07-10 17:53:36
6,0x93411f9e4078eb5a109eeb7a7996f4e62d12a65a0a97...,-10.401436416235884784,WETH,13000,CRV,21713.23977488263021007616663552795,2021-07-10 17:53:36
7,0x93411f9e4078eb5a109eeb7a7996f4e62d12a65a0a97...,-21728.390275,USDC,10.401436416235884784,WETH,21720.81502494131510503808331776398,2021-07-10 17:53:36
8,0x419654f68867596d066c593bb9ec98a0c11fe058a9e4...,-500,MM,8591.053993,USDC,8591.053993,2021-07-10 17:53:36
9,0x3049ebef9629a1a2235108d3c0469b793250b92565b7...,-7686.699146309436710399,INST,25.665802766382694243,WETH,53577.95857997341570665005557220905,2021-07-10 17:53:36
